In [ ]:
using Pkg
Pkg.activate(".")
# Pkg.instantiate()
using SymPy
using LinearAlgebra

  Activating project at `c:\Users\tim\Documents\2025examples\02-13`


In [11]:
m1,m2,k1,k2,k3 = symbols("m_1, m_2, k_1, k_2, k_3", real=true, positive=true)
c1,c2,c3,c4 = symbols("c_1, c_2, c_3, c_4", real=true);

In [ ]:
M = diagm([m1,m2]);
K = [(k1+k2) -k2;
      -k2  (k2+k3)];
A = [ zeros(2,2) diagm([1,1]);
    -M\K zeros(2,2)]

4×4 Matrix{Sym{PyCall.PyObject}}:
              0.0               0.0    1    0
              0.0               0.0    0    1
 (-k_1 - k_2)/m_1           k_2/m_1  0.0  0.0
          k_2/m_2  (-k_2 - k_3)/m_2  0.0  0.0

In [10]:
B = [ 0; 0; M\[1;0]]

4-element Vector{Sym}:
     0
     0
 1/m_1
     0

In [12]:
C = [c1 c2 c3 c4]

1×4 Matrix{Sym{PyCall.PyObject}}:
 c_1  c_2  c_3  c_4

Let's build the Observability Matrix

In [16]:
Omx = [C; C*A; C*A^2; C*A^3]
Omx = [simplify(y) for y in Omx]

4×4 Matrix{Sym{PyCall.PyObject}}:
                                                                                                       c_1  …                                            c_4
                                                            (-c_3*m_2*(k_1 + k_2) + c_4*k_2*m_1)/(m_1*m_2)                                               c_2
                                                            (-c_1*m_2*(k_1 + k_2) + c_2*k_2*m_1)/(m_1*m_2)     (c_3*k_2*m_2 - c_4*m_1*(k_2 + k_3))/(m_1*m_2)
 (c_3*m_2*(k_2^2*m_1 + m_2*(k_1 + k_2)^2) - c_4*k_2*m_1*(m_1*(k_2 + k_3) + m_2*(k_1 + k_2)))/(m_1^2*m_2^2)     (c_1*k_2*m_2 - c_2*m_1*(k_2 + k_3))/(m_1*m_2)

Let's compute when this matrix would be singular, and also let's assume that we aren't going to measure any velocities ($c_3 \to 0$, $c_4 \to 0$)

In [22]:
char1 = det(Omx) |> subs(c3=>0, c4=>0) |> simplify

     4    2        3                    3        2        3        2        3 
  c_1 *k_2    2*c_1 *c_2*k_1*k_2   2*c_1 *c_2*k_2    2*c_1 *c_2*k_2    2*c_1 *
- --------- - ------------------ + --------------- - --------------- + -------
        2               2              m_1*m_2                2             m_
     m_1             m_1                                   m_1                

                 2    2    2        2    2                2    2              
c_2*k_2*k_3   c_1 *c_2 *k_1    2*c_1 *c_2 *k_1*k_2   2*c_1 *c_2 *k_1*k_2   2*c
----------- - -------------- + ------------------- - ------------------- + ---
1*m_2                 2              m_1*m_2                    2             
                   m_1                                       m_1              

  2    2              2    2    2        2    2    2      2    2    2        2
_1 *c_2 *k_1*k_3   c_1 *c_2 *k_2    4*c_1 *c_2 *k_2    c_1 *c_2 *k_2    2*c_1 
---------------- - -------------- + --------------

Now let's solve for $c_1$ in terms of $c_2$ and the parameters of the system

In [24]:
sol1 = solve(char1, c1)

2-element Vector{Sym{PyCall.PyObject}}:
 c_2*(-k_1*m_2 + k_2*m_1 - k_2*m_2 + k_3*m_1 - sqrt(k_1^2*m_2^2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2^2 - 2*k_1*k_3*m_1*m_2 + k_2^2*m_1^2 + 2*k_2^2*m_1*m_2 + k_2^2*m_2^2 + 2*k_2*k_3*m_1^2 - 2*k_2*k_3*m_1*m_2 + k_3^2*m_1^2))/(2*k_2*m_2)
 c_2*(-k_1*m_2 + k_2*m_1 - k_2*m_2 + k_3*m_1 + sqrt(k_1^2*m_2^2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2^2 - 2*k_1*k_3*m_1*m_2 + k_2^2*m_1^2 + 2*k_2^2*m_1*m_2 + k_2^2*m_2^2 + 2*k_2*k_3*m_1^2 - 2*k_2*k_3*m_1*m_2 + k_3^2*m_1^2))/(2*k_2*m_2)

Lastly, we can simplify this further using the values we know from the example

In [31]:
[ x(k1=>1, k2=>2, k3=>1, m1=>1, m2=>1) for x in sol1 ]

2-element Vector{Sym{PyCall.PyObject}}:
 -c_2
  c_2